## Imports

In [9]:
import os
import sys

import numpy as np
import pandas as pd
from scipy.stats import randint, uniform

from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score, mean_squared_error, make_scorer
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, StratifiedKFold, KFold, train_test_split
from sklearn.preprocessing import PolynomialFeatures

dir_parts = os.getcwd().split(os.path.sep)
root_index = dir_parts.index('ML-B')
root_path = os.path.sep.join(dir_parts[:root_index + 1])
sys.path.append(root_path + '/code/')
from data.data_config import Dataset
from data.data_utils import load_monk, load_cup, store_monk_result, store_cup_result
from hyperparameter_tuning import tuning_search_top_configs
from training.metrics import mean_euclidean_error

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Random Forest
In this notebook we test a **RandomForestClassifier** and a **RandomForestRegressor** w.r.t. the tasks at hand, i.e. the three MONK's problems and the CUP dataset respectively.

In [10]:
MODEL_NAME = 'RandomForest'
INTERNAL_TEST_SPLIT = 0.1 # internal test split percentage
RANDOM_STATE = 128 # reproducibility
N_SPLITS = 5 # cross-validation
POLY_DEGREE = 3

## Path

In [11]:
# Directories
results_dir = root_path + '/results/' + MODEL_NAME

# Filepaths (MONK)
m1_dev_path, m1_test_path = Dataset.MONK_1.dev_path, Dataset.MONK_1.test_path # MONK 1
m2_dev_path, m2_test_path = Dataset.MONK_2.dev_path, Dataset.MONK_2.test_path # MONK 2
m3_dev_path, m3_test_path = Dataset.MONK_3.dev_path, Dataset.MONK_3.test_path # MONK 3

# Filepaths (CUP)
cup_dev_path, cup_test_path = Dataset.CUP.dev_path, Dataset.CUP.test_path

# MONK-1

In [4]:
# Load MONK-1
x_dev_m1, y_dev_m1, x_test_m1, y_test_m1 = load_monk(m1_dev_path, m1_test_path)

## Grid Search
Grid search is a simple hyper-parameter tuning techniques useful for finding the best configuration for a specific ML model. It involves:
- defining a grid of hparams values
- systematically test all possible combinations

Other approaches include Random search or Bayesian optimization.

In [5]:
# hparams grid for grid search
rfc_hparams_spaces = {
    'max_depth': [None, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'min_samples_split': [2, 4],
    'criterion': ['gini', 'entropy'],
    'bootstrap': [False],
}

In [6]:
# perform grid search with KFold
grid_search_m1 = GridSearchCV(
    RandomForestClassifier(random_state=128),
    param_grid=rfc_hparams_spaces,
    cv=KFold(n_splits=N_SPLITS, shuffle=True, random_state=128),
    scoring='accuracy',
    verbose=1
)

grid_search_m1.fit(x_dev_m1, y_dev_m1)

Fitting 5 folds for each of 36 candidates, totalling 180 fits


GridSearchCV(cv=KFold(n_splits=5, random_state=128, shuffle=True),
             estimator=RandomForestClassifier(random_state=128),
             param_grid={'bootstrap': [False], 'criterion': ['gini', 'entropy'],
                         'max_depth': [None, 5, 10],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 4]},
             scoring='accuracy', verbose=1)

In [7]:
best_configs_m1 = tuning_search_top_configs(grid_search_m1.cv_results_) # top k config

Model rank 1 - Config: {'bootstrap': False, 'criterion': 'entropy', 'max_depth': 5, 'min_samples_leaf': 1, 'min_samples_split': 2}
Mean score 0.9193 - Std score: 0.0438

Model rank 2 - Config: {'bootstrap': False, 'criterion': 'gini', 'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 4}
Mean score 0.9113 - Std score: 0.0530

Model rank 3 - Config: {'bootstrap': False, 'criterion': 'gini', 'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 4}
Mean score 0.9113 - Std score: 0.0530

Model rank 4 - Config: {'bootstrap': False, 'criterion': 'entropy', 'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 4}
Mean score 0.9033 - Std score: 0.0406

Model rank 5 - Config: {'bootstrap': False, 'criterion': 'gini', 'max_depth': 10, 'min_samples_leaf': 2, 'min_samples_split': 4}
Mean score 0.9033 - Std score: 0.0317



## Training - Testing

In [8]:
# Create and train the RandomForestClassifier
rfc_m1 = RandomForestClassifier(**grid_search_m1.best_params_)
rfc_m1.fit(x_dev_m1, y_dev_m1)

RandomForestClassifier(bootstrap=False, criterion='entropy', max_depth=5)

In [9]:
print('-- DEVELOPMENT --')
acc_dev_m1 = accuracy_score(y_dev_m1, rfc_m1.predict(x_dev_m1))
mse_dev_m1 = mean_squared_error(y_dev_m1, rfc_m1.predict(x_dev_m1))
print(f'MSE: {mse_dev_m1:.4f} - Accuracy: {acc_dev_m1:.4f}')

-- DEVELOPMENT --
MSE: 0.0000 - Accuracy: 1.0000


In [10]:
print('-- TEST --')
acc_test_m1 = accuracy_score(y_test_m1, rfc_m1.predict(x_test_m1))
mse_test_m1 = mean_squared_error(y_test_m1, rfc_m1.predict(x_test_m1))
print(f'MSE: {mse_test_m1:.4f} - Accuracy: {acc_test_m1:.4f}')

-- TEST --
MSE: 0.0301 - Accuracy: 0.9699


## Store results

In [11]:
report_m1 = {
    'dev': {'mse': mse_dev_m1, 'accuracy': acc_dev_m1},
    'test': {'mse': mse_test_m1, 'accuracy': acc_test_m1}
}

store_monk_result(results_dir + '/MONK1/', grid_search_m1.best_params_, report_m1)

# MONK-2

In [12]:
# Load MONK-2
x_dev_m2, y_dev_m2, x_test_m2, y_test_m2 = load_monk(m2_dev_path, m2_test_path)

## Grid Search

In [13]:
# perform grid search with KFold
grid_search_m2 = GridSearchCV(
    RandomForestClassifier(random_state=RANDOM_STATE),
    param_grid=rfc_hparams_spaces,
    cv=KFold(n_splits=N_SPLITS, shuffle=True, random_state=RANDOM_STATE),
    scoring='accuracy',
    verbose=1
)

grid_search_m2.fit(x_dev_m2, y_dev_m2)

Fitting 5 folds for each of 36 candidates, totalling 180 fits


GridSearchCV(cv=KFold(n_splits=5, random_state=128, shuffle=True),
             estimator=RandomForestClassifier(random_state=128),
             param_grid={'bootstrap': [False], 'criterion': ['gini', 'entropy'],
                         'max_depth': [None, 5, 10],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 4]},
             scoring='accuracy', verbose=1)

In [14]:
best_configs_m2 = tuning_search_top_configs(grid_search_m2.cv_results_) # top k config

Model rank 1 - Config: {'bootstrap': False, 'criterion': 'entropy', 'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 2}
Mean score 0.6332 - Std score: 0.0437

Model rank 2 - Config: {'bootstrap': False, 'criterion': 'gini', 'max_depth': 5, 'min_samples_leaf': 1, 'min_samples_split': 2}
Mean score 0.6214 - Std score: 0.0423

Model rank 3 - Config: {'bootstrap': False, 'criterion': 'gini', 'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2}
Mean score 0.6212 - Std score: 0.0511

Model rank 4 - Config: {'bootstrap': False, 'criterion': 'entropy', 'max_depth': 5, 'min_samples_leaf': 2, 'min_samples_split': 2}
Mean score 0.6212 - Std score: 0.0350

Model rank 5 - Config: {'bootstrap': False, 'criterion': 'entropy', 'max_depth': 5, 'min_samples_leaf': 2, 'min_samples_split': 4}
Mean score 0.6212 - Std score: 0.0350



## Training - Testing

In [15]:
# Create and train the RandomForestClassifier
rfc_m2 = RandomForestClassifier(**grid_search_m2.best_params_)
rfc_m2.fit(x_dev_m2, y_dev_m2)

RandomForestClassifier(bootstrap=False, criterion='entropy', max_depth=10)

In [16]:
print('-- DEVELOPMENT --')
acc_dev_m2 = accuracy_score(y_dev_m2, rfc_m2.predict(x_dev_m2))
mse_dev_m2 = mean_squared_error(y_dev_m2, rfc_m2.predict(x_dev_m2))
print(f'MSE: {mse_dev_m2:.4f} - Accuracy: {acc_dev_m2:.4f}')

-- DEVELOPMENT --
MSE: 0.0000 - Accuracy: 1.0000


In [17]:
print('-- TEST --')
acc_test_m2 = accuracy_score(y_test_m2, rfc_m2.predict(x_test_m2))
mse_test_m2 = mean_squared_error(y_test_m2, rfc_m2.predict(x_test_m2))
print(f'MSE: {mse_test_m2:.4f} - Accuracy: {acc_test_m2:.4f}')

-- TEST --
MSE: 0.2292 - Accuracy: 0.7708


## Store results

In [18]:
report_m2 = {
    'dev': {'mse': mse_dev_m2, 'accuracy': acc_dev_m2},
    'test': {'mse': mse_test_m2, 'accuracy': acc_test_m2}
}

store_monk_result(results_dir + '/MONK2/', grid_search_m2.best_params_, report_m2)

# MONK-3

In [19]:
# Load MONK-3
x_dev_m3, y_dev_m3, x_test_m3, y_test_m3 = load_monk(m3_dev_path, m3_test_path)

## Grid Search

In [20]:
# perform grid search with KFold
grid_search_m3 = GridSearchCV(
    RandomForestClassifier(random_state=RANDOM_STATE),
    param_grid=rfc_hparams_spaces,
    cv=KFold(n_splits=N_SPLITS, shuffle=True, random_state=RANDOM_STATE),
    scoring='accuracy',
    verbose=1
)

grid_search_m3.fit(x_dev_m3, y_dev_m3)

Fitting 5 folds for each of 36 candidates, totalling 180 fits


GridSearchCV(cv=KFold(n_splits=5, random_state=128, shuffle=True),
             estimator=RandomForestClassifier(random_state=128),
             param_grid={'bootstrap': [False], 'criterion': ['gini', 'entropy'],
                         'max_depth': [None, 5, 10],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 4]},
             scoring='accuracy', verbose=1)

In [21]:
best_configs_m3 = tuning_search_top_configs(grid_search_m3.cv_results_) # top k config

Model rank 1 - Config: {'bootstrap': False, 'criterion': 'gini', 'max_depth': 10, 'min_samples_leaf': 4, 'min_samples_split': 4}
Mean score 0.9343 - Std score: 0.0565

Model rank 2 - Config: {'bootstrap': False, 'criterion': 'entropy', 'max_depth': 5, 'min_samples_leaf': 4, 'min_samples_split': 4}
Mean score 0.9343 - Std score: 0.0565

Model rank 3 - Config: {'bootstrap': False, 'criterion': 'entropy', 'max_depth': 5, 'min_samples_leaf': 4, 'min_samples_split': 2}
Mean score 0.9343 - Std score: 0.0565

Model rank 4 - Config: {'bootstrap': False, 'criterion': 'entropy', 'max_depth': 5, 'min_samples_leaf': 2, 'min_samples_split': 4}
Mean score 0.9343 - Std score: 0.0565

Model rank 5 - Config: {'bootstrap': False, 'criterion': 'entropy', 'max_depth': 5, 'min_samples_leaf': 2, 'min_samples_split': 2}
Mean score 0.9343 - Std score: 0.0565



## Training - Testing

In [22]:
# Create and train the RandomForestClassifier
rfc_m3 = RandomForestClassifier(**grid_search_m3.best_params_)
rfc_m3.fit(x_dev_m3, y_dev_m3)

RandomForestClassifier(bootstrap=False, min_samples_leaf=4)

In [23]:
print('-- DEVELOPMENT --')
acc_dev_m3 = accuracy_score(y_dev_m3, rfc_m3.predict(x_dev_m3))
mse_dev_m3 = mean_squared_error(y_dev_m3, rfc_m3.predict(x_dev_m3))
print(f'MSE: {mse_dev_m3:.4f} - Accuracy: {acc_dev_m3:.4f}')

-- DEVELOPMENT --
MSE: 0.0574 - Accuracy: 0.9426


In [24]:
print('-- TEST --')
acc_test_m3 = accuracy_score(y_test_m3, rfc_m3.predict(x_test_m3))
mse_test_m3 = mean_squared_error(y_test_m3, rfc_m3.predict(x_test_m3))
print(f'MSE: {mse_test_m3:.4f} - Accuracy: {acc_test_m3:.4f}')

-- TEST --
MSE: 0.0301 - Accuracy: 0.9699


## Store results

In [25]:
report_m3 = {
    'dev': {'mse': mse_dev_m3, 'accuracy': acc_dev_m3},
    'test': {'mse': mse_test_m3, 'accuracy': acc_test_m3}
}

store_monk_result(results_dir + '/MONK3/', grid_search_m3.best_params_, report_m3)

# CUP

In [15]:
# Load CUP
x_dev_cup, y_dev_cup, x_test_cup = load_cup(cup_dev_path, cup_test_path)

## Dev - Internal Test Split 
The development dataset is split between training and internal test ($90-10$).

In [16]:
# Split the internal test set
x_train_cup, x_internal_test_cup, y_train_cup, y_internal_test_cup = train_test_split(
    x_dev_cup, y_dev_cup, test_size=0.1, random_state=128
)

## Polynomial Features pre-processing

In [28]:
# --- COMMENT TO USE NON-ENCHANED DATASET --- 
# Polynomial features pre-processing
poly = PolynomialFeatures(degree=POLY_DEGREE)
x_train_cup = poly.fit_transform(x_train_cup)
x_internal_test_cup = poly.transform(x_internal_test_cup)
x_test_cup = poly.transform(x_test_cup)

## Hyper-paramater Tuning

### Grid Search

In [29]:
rfr_hparams_spaces_gridsearch = {
    'n_estimators': [100, 200],
    'max_features': ['sqrt'],
    'max_depth': [5, 10, 20, 50, None],
    'min_samples_split': [2, 4],
    'min_samples_leaf': [1, 2, 4],
    'criterion': ['squared_error'],
    'bootstrap': [False, True],
}

In [30]:
grid_search_cup = GridSearchCV(
    RandomForestRegressor(random_state=RANDOM_STATE), 
    rfr_hparams_spaces_gridsearch, 
    cv=KFold(n_splits=N_SPLITS, shuffle=True, random_state=RANDOM_STATE),
    scoring=make_scorer(mean_euclidean_error, greater_is_better=False),
    verbose=1
)

grid_search_cup.fit(x_train_cup, y_train_cup)

Fitting 5 folds for each of 2 candidates, totalling 10 fits


GridSearchCV(cv=KFold(n_splits=5, random_state=128, shuffle=True),
             estimator=RandomForestRegressor(random_state=128),
             param_grid={'bootstrap': [False, True],
                         'criterion': ['squared_error'], 'max_depth': [20],
                         'max_features': ['sqrt'], 'min_samples_leaf': [1],
                         'min_samples_split': [2], 'n_estimators': [200]},
             scoring=make_scorer(mean_euclidean_error, greater_is_better=False),
             verbose=1)

In [31]:
best_configs_cup = tuning_search_top_configs(grid_search_cup.cv_results_) # top k config

Model rank 1 - Config: {'bootstrap': False, 'criterion': 'squared_error', 'max_depth': 20, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
Mean score -2.3427 - Std score: 0.1891

Model rank 2 - Config: {'bootstrap': True, 'criterion': 'squared_error', 'max_depth': 20, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
Mean score -2.6853 - Std score: 0.2209



### Random Search

In [31]:
best_params = grid_search_cup.best_params_
print(best_params)

{'bootstrap': False, 'criterion': 'squared_error', 'max_depth': 20, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}


In [20]:
# Define new parameter space for RandomizedSearchCV around the best parameters
rfr_hparams_spaces_randomsearch = {
    'n_estimators': randint(best_params['n_estimators'] - 50, best_params['n_estimators'] + 50),
    'max_features': [best_params['max_features']],
    'max_depth': [best_params['max_depth']],
    'min_samples_split': randint(max(2, best_params['min_samples_split'] - 2), best_params['min_samples_split'] + 3),
    'min_samples_leaf': randint(max(1, best_params['min_samples_leaf'] - 1), best_params['min_samples_leaf'] + 2),
    'criterion': [best_params['criterion']],
    'bootstrap': [best_params['bootstrap']],
    # 'ccp_alpha': [best_params['ccp_alpha']]
}

rfr_hparams_spaces_randomsearch = {
    'n_estimators': [232],
    'max_features': ['sqrt'],
    'max_depth': [20],
    'min_samples_split': [3],
    'min_samples_leaf': [1],
    'criterion': ['squared_error'],
    'bootstrap': [False],
}

In [33]:
random_search_cup = RandomizedSearchCV(
    RandomForestRegressor(random_state=RANDOM_STATE), 
    rfr_hparams_spaces_randomsearch, 
    n_iter=20,
    cv=KFold(n_splits=N_SPLITS, shuffle=True, random_state=RANDOM_STATE),
    scoring=make_scorer(mean_euclidean_error, greater_is_better=False),
    verbose=1,
    random_state=RANDOM_STATE
)

random_search_cup.fit(x_train_cup, y_train_cup)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


RandomizedSearchCV(cv=KFold(n_splits=5, random_state=128, shuffle=True),
                   estimator=RandomForestRegressor(random_state=128), n_iter=1,
                   param_distributions={'bootstrap': [False],
                                        'criterion': ['squared_error'],
                                        'max_depth': [20],
                                        'max_features': ['sqrt'],
                                        'min_samples_leaf': [1],
                                        'min_samples_split': [3],
                                        'n_estimators': [232]},
                   random_state=128,
                   scoring=make_scorer(mean_euclidean_error, greater_is_better=False),
                   verbose=1)

In [34]:
tuning_search_top_configs(random_search_cup.cv_results_)

Model rank 1 - Config: {'n_estimators': 232, 'min_samples_split': 3, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 20, 'criterion': 'squared_error', 'bootstrap': False}
Mean score -2.4241 - Std score: 0.1966



({1: {'config': {'n_estimators': 232,
    'min_samples_split': 3,
    'min_samples_leaf': 1,
    'max_features': 'sqrt',
    'max_depth': 20,
    'criterion': 'squared_error',
    'bootstrap': False},
   'Mean score': -2.4241,
   'Std score': 0.1966}},
 [0])

In [40]:
best_score_grid = grid_search_cup.best_score_
best_score_random = random_search_cup.best_score_

# Check if best result is from GridSearc or RandomSearch
if best_score_random > best_score_grid:
    print("Best configuration from RandomizedSearch:\n")
    best_hparams = random_search_cup.best_params_
    print(best_hparams)
else:
    best_hparams = grid_search_cup.best_params_
    print("GridSearchCV resulted in the best configuration.")

GridSearchCV resulted in the best configuration.


## Training - Testing

In [42]:
# Create a RandomForestRegressor with the best hparams
rfr = RandomForestRegressor(**best_hparams)

# Train the model
rfr.fit(x_train_cup, y_train_cup)

RandomForestRegressor(bootstrap=False, max_depth=20, max_features='sqrt',
                      n_estimators=200)

In [43]:
print('-- TRAINING --')
mee_train_cup = mean_euclidean_error(y_train_cup, rfr.predict(x_train_cup))
mse_train_cup = mean_squared_error(y_train_cup, rfr.predict(x_train_cup))
print(f'Loss (MSE): {mse_train_cup:.4f} - MEE: {mee_train_cup:.4f}')

-- TRAINING --
Loss (MSE): 0.0000 - MEE: 0.0000


In [44]:
print('-- INTERNAL TEST --')
mee_internal_test_cup = mean_euclidean_error(y_internal_test_cup, rfr.predict(x_internal_test_cup))
mse_internal_test_cup = mean_squared_error(y_internal_test_cup, rfr.predict(x_internal_test_cup))
print(f'Loss (MSE): {mse_internal_test_cup:.4f} - MEE: {mee_internal_test_cup:.4f}')

-- INTERNAL TEST --
Loss (MSE): 3.4108 - MEE: 2.3534


In [45]:
# Blind test set predictions
blind_test_preds_cup = rfr.predict(x_test_cup)

## Store results

In [46]:
report_cup = {
    'train': {'mse': mse_train_cup, 'mee': mee_train_cup},
    'internal_test': {'mse': mse_internal_test_cup, 'mee': mee_internal_test_cup},
}

store_cup_result(results_dir + '/CUP/', best_hparams, report_cup, blind_test_preds_cup, is_poly=True)